In [2]:
import pandas as pd
import numpy as np
from urllib.request import urlopen
import requests
from bs4 import BeautifulSoup

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

### 공공데이터포털>도로교통공단_보행노인사고다발지역정보서비스(https://www.data.go.kr/data/15057666/openapi.do) 에서
### api활용신청 후 필요한 데이터 크로링하여 df로 저장


## 노인교통사고사망자다발지역 데이터프레임으로 저장 - 충청남도

In [3]:
# url 및 서비스 key
# endpoint 뒤에 항상 ? 붙여야함
# 홈페이지에 들어가보면 endpoint가 오류가 있음을 확인
# http://apis.data.go.kr/B552061/frequentzoneOldman => http://apis.data.go.kr/B552061/frequentzoneOldman/getRestFrequentzoneOldman

endpoint = 'http://apis.data.go.kr/B552061/frequentzoneOldman/getRestFrequentzoneOldman?'
serviceKey = "k1ydVK60GxfNLgnjRUVD2jHYPBs38CyflSPrYbC5CP5gdftFpdrBhdy%2F8YlxDfMfAAq05iKqUFORmAoBJOLAxA%3D%3D"

In [4]:
# 충남,전남,제주의 시도코드와 구군코드를 엑셀로 정리하여 불러오기
siDo_guGun_code = pd.read_csv('../[0]data/전남_충남_제주_시군구코드.csv')
siDo_guGun_code

,전라남도,충청남도,제주도
0,46,44.0,50.0
1,810,250.0,110.0
2,770,150.0,130.0
3,720,710.0,0.0
4,230,230.0,NaN
5,730,830.0,NaN
6,170,270.0,NaN
7,710,180.0,NaN
8,110,760.0,NaN
9,840,210.0,NaN


In [5]:
# siDo_guGun_code에서 충청남도 행만 추출하여 siDoguGunCode_chungnam에 저장
siDoguGunCode_chungnam = siDo_guGun_code[['충청남도']]
siDoguGunCode_chungnam

,충청남도
0,44.0
1,250.0
2,150.0
3,710.0
4,230.0
5,830.0
6,270.0
7,180.0
8,760.0
9,210.0


In [6]:
# NaN값이 들어간 행 삭제 후 원본반영
siDoguGunCode_chungnam.dropna(inplace=True)

<ipython-input-6-600dc4a8403e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  siDoguGunCode_chungnam.dropna(inplace=True)


In [7]:
siDoguGunCode_chungnam

,충청남도
0,44.0
1,250.0
2,150.0
3,710.0
4,230.0
5,830.0
6,270.0
7,180.0
8,760.0
9,210.0


In [8]:
# siDoguGunCode_chungnam 내부의 값들의 int로 형변환 (소수점 없애기) 
siDoguGunCode_chungnam = siDoguGunCode_chungnam.astype('int')

In [9]:
# siDoguGunCode_chungnam 내부의 값들의 문자형으로 형변환 (뒤에서 paramset에 문자열로 넣어야하기 때문에) 
siDoguGunCode_chungnam = siDoguGunCode_chungnam.astype('str')
siDoguGunCode_chungnam.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20 entries, 0 to 19
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   충청남도    20 non-null     object
dtypes: object(1)
memory usage: 320.0+ bytes


In [10]:
# siDoguGunCode_chungnam에서 충청남도행을 리스트형태로 변환
siDoguGunCode_chungnam['충청남도'].to_list()

['44',
 '250',
 '150',
 '710',
 '230',
 '830',
 '270',
 '180',
 '760',
 '210',
 '770',
 '200',
 '730',
 '810',
 '130',
 '131',
 '133',
 '790',
 '825',
 '800']

In [11]:
searchYearCd = '2019'

# siDoguGunCode_chungnam의 0번째 행은 시도코드
siDo = siDoguGunCode_chungnam['충청남도'][0]

# siDoguGunCode_chungnam의 나머지 행은 구군코드
guGun = siDoguGunCode_chungnam['충청남도'][1:]
type_ = "xml"
numOfRows = '10'
pageNo = '1'

In [12]:
df_chungnam = pd.DataFrame()

for gu in guGun:
    paramset = "serviceKey=" + serviceKey +"&" \
                +"searchYearCd=" + searchYearCd +"&" \
                +"siDo=" + siDo +"&" \
                +"guGun=" + gu +"&" \
                +"type=" + type_ +"&" \
                +"numOfRows=" + numOfRows +"&" \
                +"pageNo=" + pageNo
    
    url = endpoint + paramset
    
    result = requests.get(url)
    bs_obj = BeautifulSoup(result.content,'html.parser')
    
    lo = bs_obj.findAll('lo_crd')
    la = bs_obj.findAll('la_crd')
    info = bs_obj.findAll('spot_nm')
    
    lat_chungnam=[]
    lng_chungnam=[]
    info_chungnam=[]
    
    for i in la:
        lat_chungnam.append(i.text)    
    for i in lo:
        lng_chungnam.append(i.text)
    for i in info:
        info_chungnam.append(i.text)
    
    location = {'장소':info_chungnam,'lat':lat_chungnam,'lng':lng_chungnam}
    chungnam_location = pd.DataFrame(location)
    df_chungnam = df_chungnam.append(chungnam_location)

In [13]:
df_chungnam.index=range(0,len(df_chungnam))
df_chungnam

,장소,lat,lng
0,충청남도 공주시 산성동(중앙귀금속 부근),36.458435308144,127.124723622698
1,충청남도 논산시 화지동(대화로51 부근),36.207532996352,127.082648275099
2,충청남도 당진시 읍내리(당진분식입구 부근),36.892076716568,126.629385952001
3,충청남도 보령시 동대동(녹십초키토산 부근),36.342855941549,126.603716273688
4,충청남도 보령시 대천동(경남사거리 부근),36.349879867962,126.594405996341
5,충청남도 부여군 부여읍(코롱얼음 부근),36.281671366541,126.909747894751
6,충청남도 서산시 동문동(아시아마트 부근),36.780848414823,126.457738983768
7,충청남도 서산시 동문동(고운로147 부근),36.78334666132,126.451857689182
8,충청남도 서천군 서천읍(서천읍군사리68011 부근),36.075234409142,126.687493994853
9,충청남도 서천군 서천읍(정금당입구 부근),36.077188365252,126.691037372278


In [14]:
df_chungnam.to_csv('../[0]data/0-7_충남_노인교통사고_사망자다발지역(결과).csv')

## 노인교통사고사망자다발지역 데이터프레임으로 저장 - 전라남도

In [15]:
siDoguGunCode_jeonnam = siDo_guGun_code[['전라남도']]
siDoguGunCode_jeonnam.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   전라남도    23 non-null     int64
dtypes: int64(1)
memory usage: 312.0 bytes


In [16]:
# siDoguGunCode_jeonnam 내부의 값들의 문자형으로 형변환 (뒤에서 paramset에 문자열로 넣어야하기 때문에) 
siDoguGunCode_jeonnam = siDoguGunCode_jeonnam.astype('str')

In [17]:
# siDoguGunCode_jeonnam에서 충청남도행을 리스트형태로 변환
siDoguGunCode_jeonnam['전라남도'].to_list()

['46',
 '810',
 '770',
 '720',
 '230',
 '730',
 '170',
 '710',
 '110',
 '840',
 '780',
 '150',
 '910',
 '130',
 '870',
 '830',
 '890',
 '880',
 '800',
 '900',
 '860',
 '820',
 '790']

In [18]:
searchYearCd = '2019'

# siDoguGunCode_jeonnam의 0번째 행은 시도코드
siDo = siDoguGunCode_jeonnam['전라남도'][0]

# siDoguGunCode_jeonnam의 나머지 행은 시도코드
guGun = siDoguGunCode_jeonnam['전라남도'][1:]
type_ = "xml"
numOfRows = '10'
pageNo = '1'

In [19]:
df_jeonnam = pd.DataFrame()
for gu in guGun:
    paramset = "serviceKey=" + serviceKey +"&" \
                +"searchYearCd=" + searchYearCd +"&" \
                +"siDo=" + siDo +"&" \
                +"guGun=" + gu +"&" \
                +"type=" + type_ +"&" \
                +"numOfRows=" + numOfRows +"&" \
                +"pageNo=" + pageNo
    
    url = endpoint + paramset
    
    result = requests.get(url)
    bs_obj = BeautifulSoup(result.content,'html.parser')
    
    lo = bs_obj.findAll('lo_crd')
    la = bs_obj.findAll('la_crd')
    info = bs_obj.findAll('spot_nm')
    
    lat_jeonnam=[]
    lng_jeonnam=[]
    info_jeonnam=[]
    
    for i in la:
        lat_jeonnam.append(i.text)    
    for i in lo:
        lng_jeonnam.append(i.text)
    for i in info:
        info_jeonnam.append(i.text)
    
    location = {'장소':info_jeonnam,'lat':lat_jeonnam,'lng':lng_jeonnam}
    jeonnam_location = pd.DataFrame(location)
    df_jeonnam = df_jeonnam.append(jeonnam_location)

In [20]:
df_jeonnam.index=range(0,len(df_jeonnam))
df_jeonnam.head()

,장소,lat,lng
0,전라남도 고흥군 고흥읍(소매점입구 부근),34.607685320422,127.281436313236
1,전라남도 구례군 구례읍(구례읍봉북리854 부근),35.21099664458,127.464026328353
2,전라남도 나주시 이창동(이창1길1 부근),34.996120977002,126.714191912128
3,전라남도 목포시 산정동(산정로1201 부근),34.799235324252,126.392500779389
4,전라남도 목포시 대성동(청호로571 부근),34.798277361912,126.382720273523


In [21]:
df_jeonnam.to_csv('../[0]data/0-7_전남_노인교통사고_사망자다발지역(결과).csv')

## 노인교통사고사망자다발지역 데이터프레임으로 저장 - 제주도

In [22]:
siDoguGunCode_jeju = siDo_guGun_code[['제주도']]
siDoguGunCode_jeju.info()
siDoguGunCode_jeju

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23 entries, 0 to 22
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   제주도     4 non-null      float64
dtypes: float64(1)
memory usage: 312.0 bytes


,제주도
0,50.0
1,110.0
2,130.0
3,0.0
4,NaN
5,NaN
6,NaN
7,NaN
8,NaN
9,NaN


In [23]:
# NaN값이 들어간 행 삭제 후 원본반영
siDoguGunCode_jeju.dropna(inplace=True)

<ipython-input-23-d973a293b0b1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  siDoguGunCode_jeju.dropna(inplace=True)


In [24]:
# siDoguGunCode_jeju 내부의 값들의 int형으로 형변환 (소수점 없애기) 
siDoguGunCode_jeju = siDoguGunCode_jeju.astype('int')

# siDoguGunCode_jeju 내부의 값들의 문자형으로 형변환 (뒤에서 paramset에 문자열로 넣어야하기 때문에) 
siDoguGunCode_jeju = siDoguGunCode_jeju.astype('str')

In [25]:
siDoguGunCode_jeju['제주도'].to_list()

['50', '110', '130', '0']

In [26]:
searchYearCd = '2019'

# siDoguGunCode_jeju의 0번째 행은 시도코드
siDo = siDoguGunCode_jeju['제주도'][0]

# siDoguGunCode_jeju의 나머지 행은 시도코드
guGun = siDoguGunCode_jeju['제주도'][1:]
type_ = "xml"
numOfRows = '10'
pageNo = '1'

In [27]:
df_jeju = pd.DataFrame()
for gu in guGun:
    paramset = "serviceKey=" + serviceKey +"&" \
                +"searchYearCd=" + searchYearCd +"&" \
                +"siDo=" + siDo +"&" \
                +"guGun=" + gu +"&" \
                +"type=" + type_ +"&" \
                +"numOfRows=" + numOfRows +"&" \
                +"pageNo=" + pageNo
    
    url = endpoint + paramset
    
    result = requests.get(url)
    bs_obj = BeautifulSoup(result.content,'html.parser')
    
    lo = bs_obj.findAll('lo_crd')
    la = bs_obj.findAll('la_crd')
    info = bs_obj.findAll('spot_nm')
    
    lat_jeju=[]
    lng_jeju=[]
    info_jeju=[]
    
    for i in la:
        lat_jeju.append(i.text)
    for i in lo:
        lng_jeju.append(i.text)
    for i in info:
        info_jeju.append(i.text)
    
    location = {'장소':info_jeju,'lat':lat_jeju,'lng':lng_jeju}
    jeju_location = pd.DataFrame(location)
    df_jeju = df_jeju.append(jeju_location)

In [28]:
df_jeju.index=range(0,len(df_jeju))
df_jeju

,장소,lat,lng
0,제주특별자치도 제주시 이도일동(스타벅스제주칠성점 부근),33.512233365978,126.525242465269
1,제주특별자치도 제주시 삼도일동(오라교 부근),33.500013039871,126.515328585447
2,제주특별자치도 제주시 일도이동(제주일도대림2차아파트 부근),33.50021902329,126.549327360071
3,제주특별자치도 제주시 노형동(제주양돈농협 부근),33.488325647512,126.477135194735
4,제주특별자치도 제주시 일도일동(동문로터리 부근),33.512804350042,126.528852200969
5,제주특별자치도 제주시 일도이동(호남석재사거리 부근),33.508198240158,126.533649947388
6,제주특별자치도 제주시 용담이동(용문로100 부근),33.505919018149,126.508640934433
7,제주특별자치도 제주시 이도이동(영도갤럭시타운 부근),33.496531829881,126.532139396617
8,제주특별자치도 서귀포시 서귀동(중앙로타리 부근),33.252987954531,126.561120256396
9,제주특별자치도 서귀포시 동홍동(동홍중앙로2 부근),33.251949532546,126.574322615198


In [29]:
df_jeju.to_csv('../[0]data/0-7_제주_노인교통사고_사망자다발지역(결과).csv')